# Chapter 10
## Data Aggregation and Group Operation

## 10.1 GroupBy Mechanics

In [1]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'key1': list('aabba'),
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,data1,data2,key1,key2
0,-1.016896,-0.203930,a,one
1,0.377984,-0.917392,a,two
2,-1.507993,1.591573,b,one
3,0.371647,-0.835871,b,two
4,0.140087,0.967515,a,one


In [2]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [3]:
grouped.mean()

key1
a   -0.166275
b   -0.568173
Name: data1, dtype: float64

In [4]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one    -0.438405
      two     0.377984
b     one    -1.507993
      two     0.371647
Name: data1, dtype: float64

In [5]:
means.unstack()

key2,one,two
key1,,
a,-0.438405,0.377984
b,-1.507993,0.371647


In [6]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.377984
            2006   -1.507993
Ohio        2005   -0.322624
            2006    0.140087
Name: data1, dtype: float64

In [7]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.166275,-0.051269
b,-0.568173,0.377851


In [8]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -0.438405  0.381792
     two   0.377984 -0.917392
b    one  -1.507993  1.591573
     two   0.371647 -0.835871

In [9]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [10]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
      data1     data2 key1 key2
0 -1.016896 -0.203930    a  one
1  0.377984 -0.917392    a  two
4  0.140087  0.967515    a  one
b
      data1     data2 key1 key2
2 -1.507993  1.591573    b  one
3  0.371647 -0.835871    b  two


In [11]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
      data1     data2 key1 key2
0 -1.016896 -0.203930    a  one
4  0.140087  0.967515    a  one
('a', 'two')
      data1     data2 key1 key2
1  0.377984 -0.917392    a  two
('b', 'one')
      data1     data2 key1 key2
2 -1.507993  1.591573    b  one
('b', 'two')
      data1     data2 key1 key2
3  0.371647 -0.835871    b  two


In [12]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,data1,data2,key1,key2
2,-1.507993,1.591573,b,one
3,0.371647,-0.835871,b,two


In [13]:
df.dtypes

data1    float64
data2    float64
key1      object
key2      object
dtype: object

In [14]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group, in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -1.016896 -0.203930
1  0.377984 -0.917392
2 -1.507993  1.591573
3  0.371647 -0.835871
4  0.140087  0.967515
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [15]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
# ... are equivalent to:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

In [16]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.381792
     two  -0.917392
b    one   1.591573
     two  -0.835871

In [17]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [18]:
s_grouped.mean()

key1  key2
a     one     0.381792
      two    -0.917392
b     one     1.591573
      two    -0.835871
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [19]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=list('abcde'),
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.876341,-1.576908,1.006737,-0.504318,-0.100609
Steve,0.140701,-1.183002,0.471826,0.402394,-1.830189
Wes,-0.630638,NaN,NaN,1.780484,0.178576
Jim,-0.965959,-1.637721,0.155504,-0.068590,1.600027
Travis,-0.302388,0.447344,2.256865,-1.092137,0.178684


In [20]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'} # unused grouping keys are OK
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.502419,-2.553857
Steve,0.874220,-2.872490
Wes,1.780484,-0.452062
Jim,0.086914,-1.003653
Travis,1.164728,0.323641


In [21]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [22]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [23]:
people

,a,b,c,d,e
Joe,-0.876341,-1.576908,1.006737,-0.504318,-0.100609
Steve,0.140701,-1.183002,0.471826,0.402394,-1.830189
Wes,-0.630638,NaN,NaN,1.780484,0.178576
Jim,-0.965959,-1.637721,0.155504,-0.068590,1.600027
Travis,-0.302388,0.447344,2.256865,-1.092137,0.178684


In [24]:
people.groupby(len).sum()

,a,b,c,d,e
3,-2.472938,-3.214628,1.162241,1.207575,1.677995
5,0.140701,-1.183002,0.471826,0.402394,-1.830189
6,-0.302388,0.447344,2.256865,-1.092137,0.178684


In [25]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.876341 -1.576908  1.006737 -0.504318 -0.100609
  two -0.965959 -1.637721  0.155504 -0.068590  1.600027
5 one  0.140701 -1.183002  0.471826  0.402394 -1.830189
6 two -0.302388  0.447344  2.256865 -1.092137  0.178684

### Grouping by Index Levels

In [26]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -1.190695 -0.109375  0.518957  1.311269  0.377717
1      2.793725  0.439996 -1.063149  1.524669 -0.549327
2     -0.676910  1.808905 -0.183634  0.140485  1.489280
3      0.279611  0.318608  0.351773 -1.619801  0.032775

In [27]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

In [28]:
df

,data1,data2,key1,key2
0,-1.016896,-0.203930,a,one
1,0.377984,-0.917392,a,two
2,-1.507993,1.591573,b,one
3,0.371647,-0.835871,b,two
4,0.140087,0.967515,a,one


In [29]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    0.330405
b    0.183683
Name: data1, dtype: float64

In [30]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.39488,1.884907
b,1.87964,2.427444


In [31]:
grouped['data1'].agg(peak_to_peak)

key1
a    1.39488
b    1.87964
Name: data1, dtype: float64

In [32]:
grouped.aggregate(peak_to_peak)['data1']

key1
a    1.39488
b    1.87964
Name: data1, dtype: float64

In [33]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.166275  0.746201 -1.016896 -0.438405  0.140087  0.259035   
b      2.0 -0.568173  1.329106 -1.507993 -1.038083 -0.568173 -0.098263   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.377984   3.0 -0.051269  0.951681 -0.917392 -0.560661 -0.203930   
b     0.371647   2.0  0.377851  1.716462 -0.835871 -0.229010  0.377851   

                          
           75%       max  
key1                      
a     0.381792  0.967515  
b     0.984712  1.591573

### Column-Wise and Multiple Function Application

In [34]:
tips = pd.read_csv('examples/tips.csv')

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [35]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.mean()

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [36]:
grouped_pct.agg('mean') # equivalent

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [37]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [38]:
peak_to_peak.__name__

'peak_to_peak'

In [39]:
(lambda x: x + 1).__name__ # lambdas will be labelled with '<lambda>' by default

'<lambda>'

In [40]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)]) # (name, function) two-tuples

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [41]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [42]:
pd.concat([grouped['tip_pct'].agg(functions), grouped['total_bill'].agg(functions)], keys=['tip_pct', 'total_bill'], axis='columns') # equivalent

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [43]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [44]:
ftuples = [('Durchshnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct             total_bill            
            Durchshnitt Abweichung Durchshnitt  Abweichung
day  smoker                                               
Fri  No        0.151650   0.000791   18.420000   25.596333
     Yes       0.174783   0.002631   16.813333   82.562438
Sat  No        0.158048   0.001581   19.661778   79.908965
     Yes       0.147906   0.003767   21.276667  101.387535
Sun  No        0.160113   0.001793   20.506667   66.099980
     Yes       0.187250   0.023757   24.120000  109.046044
Thur No        0.160298   0.001503   17.113111   59.625081
     Yes       0.163863   0.001551   19.190588   69.808518

In [45]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [46]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### Returning Aggregated Data Without Row Indexes

In [47]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [48]:
tips.groupby(['day', 'smoker']).mean().reset_index() # equivalent but slower

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


## 10.3 Apply: General split-apply-combine

In [49]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [50]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [51]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [52]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [53]:
tips.groupby('smoker')[['tip_pct']].apply(lambda x: x.describe()).unstack() # equivalent 

tip_pct                                                              \
         count      mean       std       min       25%       50%       75%   
smoker                                                                       
No       151.0  0.159328  0.039910  0.056797  0.136906  0.155625  0.185014   
Yes       93.0  0.163196  0.085119  0.035638  0.106771  0.153846  0.195059   

                  
             max  
smoker            
No      0.291990  
Yes     0.710345

In [54]:
result.columns.name = 'tip_pct'
result

tip_pct,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [55]:
result.unstack('smoker')

tip_pct  smoker
count    No        151.000000
         Yes        93.000000
mean     No          0.159328
         Yes         0.163196
std      No          0.039910
         Yes         0.085119
min      No          0.056797
         Yes         0.035638
25%      No          0.136906
         Yes         0.106771
50%      No          0.155625
         Yes         0.153846
75%      No          0.185014
         Yes         0.195059
max      No          0.291990
         Yes         0.710345
dtype: float64

### Suppressing the Group Keys

In [56]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [57]:
tips.groupby('smoker', as_index=False).apply(top)

total_bill   tip smoker   day    time  size   tip_pct
0 88        24.71  5.85     No  Thur   Lunch     2  0.236746
  185       20.69  5.00     No   Sun  Dinner     5  0.241663
  51        10.29  2.60     No   Sun  Dinner     2  0.252672
  149        7.51  2.00     No  Thur   Lunch     2  0.266312
  232       11.61  3.39     No   Sat  Dinner     2  0.291990
1 109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
  183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
  67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
  178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
  172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [58]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### Quantile and Bucket Analysis

In [59]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0      (0.256, 1.894]
1     (-1.383, 0.256]
2     (-1.383, 0.256]
3     (-1.383, 0.256]
4     (-1.383, 0.256]
5      (0.256, 1.894]
6      (0.256, 1.894]
7    (-3.027, -1.383]
8     (-1.383, 0.256]
9     (-1.383, 0.256]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.027, -1.383] < (-1.383, 0.256] < (0.256, 1.894] < (1.894, 3.532]]

In [60]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}
grouped = frame.data1.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.027, -1.383]",84.0,-1.382796,-1.959566,-3.020857
"(-1.383, 0.256]",514.0,0.253822,-0.411854,-1.373996
"(0.256, 1.894]",372.0,1.881344,0.895334,0.258103
"(1.894, 3.532]",30.0,3.532456,2.361122,1.917919


In [61]:
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.027, -1.383]",84.0,2.925418,0.117759,-3.104550
"(-1.383, 0.256]",514.0,3.256848,0.052754,-2.560610
"(0.256, 1.894]",372.0,3.413774,-0.067227,-3.170069
"(1.894, 3.532]",30.0,2.022458,-0.061097,-2.231951


In [62]:
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.925418,0.118644,-3.104550
1,100.0,2.072115,-0.016982,-2.081576
2,100.0,2.011483,0.017540,-2.560610
3,100.0,3.256848,0.140297,-2.142303
4,100.0,2.865166,0.019888,-2.450942
5,100.0,2.625640,0.091558,-2.083588
6,100.0,3.385396,-0.159921,-3.048233
7,100.0,1.809498,-0.023851,-2.058416
8,100.0,3.413774,-0.001370,-3.170069


### Example: Filling Missing Values with Group-Specific Values

In [63]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -2.136955
2         NaN
3    1.428612
4         NaN
5   -0.485034
dtype: float64

In [64]:
s.fillna(s.mean())

0   -0.397792
1   -2.136955
2   -0.397792
3    1.428612
4   -0.397792
5   -0.485034
dtype: float64

In [65]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.942124
New York      0.204245
Vermont       1.226632
Florida       0.006912
Oregon       -1.434500
Nevada       -0.453538
California    0.614160
Idaho        -1.563646
dtype: float64

In [66]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.942124
New York      0.204245
Vermont            NaN
Florida       0.006912
Oregon       -1.434500
Nevada             NaN
California    0.614160
Idaho              NaN
dtype: float64

In [67]:
data.groupby(group_key).mean()

East    0.384427
West   -0.410170
dtype: float64

In [68]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.942124
New York      0.204245
Vermont       0.384427
Florida       0.006912
Oregon       -1.434500
Nevada       -0.410170
California    0.614160
Idaho        -0.410170
dtype: float64

In [69]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio          0.942124
New York      0.204245
Vermont       0.500000
Florida       0.006912
Oregon       -1.434500
Nevada       -1.000000
California    0.614160
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [70]:
suits = list('HSCD')
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + list('JKQ')
cards = []
for suit in suits:
    cards.extend(str(base_name) + suit for base_name in base_names)

deck = pd.Series(card_val, index=cards)
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [71]:
def draw(deck, n=5):
    return deck.sample(n)

draw(deck)

3C     3
JC    10
8H     8
5D     5
6C     6
dtype: int64

In [72]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  8C     8
   2C     2
D  5D     5
   8D     8
H  5H     5
   2H     2
S  JS    10
   AS     1
dtype: int64

In [73]:
# alternatively:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

5C      5
8C      8
QD     10
10D    10
3H      3
4H      4
6S      6
QS     10
dtype: int64

### Example: Group Weighted Average and Correlation

In [74]:
df = pd.DataFrame({'category': list('aaaabbbb'),
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,-1.216289,0.456096
1,a,1.096017,0.239257
2,a,2.021991,0.587312
3,a,1.353887,0.385363
4,b,1.069421,0.250089
5,b,0.268124,0.181782
6,b,-1.240700,0.566113
7,b,-0.023188,0.929607


In [75]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.849364
b   -0.211529
dtype: float64

In [76]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [77]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [78]:
spx_corr = lambda x: x.corrwith(x['SPX'])
rets = close_px.pct_change().dropna()
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [79]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In [80]:
import statsmodels.api as sm

def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y, X).fit()
    return result.params

by_year.apply(regress, 'AAPL', ['SPX'])

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 Pivot Tables and Cross-Tabulation

In [81]:
tips.pivot_table(index=['day', 'smoker']) # applies mean aggregation by default

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [82]:
tips.groupby(['day', 'smoker']).mean() # equivalent (different column ordering only)

total_bill       tip      size   tip_pct
day  smoker                                          
Fri  No       18.420000  2.812500  2.250000  0.151650
     Yes      16.813333  2.714000  2.066667  0.174783
Sat  No       19.661778  3.102889  2.555556  0.158048
     Yes      21.276667  2.875476  2.476190  0.147906
Sun  No       20.506667  3.167895  2.929825  0.160113
     Yes      24.120000  3.516842  2.578947  0.187250
Thur No       17.113111  2.673778  2.488889  0.160298
     Yes      19.190588  3.030000  2.352941  0.163863

In [83]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'])

size   tip_pct
time   day                     
Dinner Fri   2.166667  0.158916
       Sat   2.517241  0.153152
       Sun   2.842105  0.166897
       Thur  2.000000  0.159744
Lunch  Fri   2.000000  0.188765
       Thur  2.459016  0.161301

In [84]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [85]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [86]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', margins=True, aggfunc=len) # cross-tabluation

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [87]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

### Cross-Tabulations: Crosstab

In [88]:
tips[:10]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [89]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244